# Краткое описание

Реализована создание графа процесса, матрица вероятностных переходов и отрисовка графа

In [40]:
def rate(self,rate1,rate2):
    pass

считает формулу (1)

на вход подаётся количество случаев когда стадия A шла после стадии B и наоборот. Это можно посчитать с помощью функции 

In [41]:
def numbers_of_cases(self,stage1,stage2):
    pass

где stage1,stage2 название стадий. К примеру, 'Stage_0', 'Stage_1'

In [42]:
def get_a_ensue_b_and_c(self,stage0,stage1,stage2):
    pass

считает формулу (4)

на вход подаётся названия стадий 

# Проблема

Как определить в каком отношении находятся стадии графа

Переход от графа процесса к граффу стадий процесса.

# Реализация

In [3]:
import numpy as np
import pandas as pd
from itertools import product as conbinations
import matplotlib.pyplot as plt
from graphviz import Digraph
from pm4py.objects.conversion.log import factory as conversion_factory  # преобразование DataFrame в лог-файл
from pm4py.algo.discovery.heuristics import factory as heuristics_miner
from pm4py.visualization.heuristics_net import factory as hn_vis_factory  # отрисовка heu_net
from pm4py.algo.discovery.alpha import factory as alpha_miner

In [4]:
#df=pd.read_csv('dataset_el.csv')
df=pd.read_csv('synthetic_process_mining_dataset5.csv')

print(df.shape)
df.head()

(4461, 3)


,id,stages,dt
0,0.0,Stage_0,2019-12-01
1,0.0,Stage_3,2019-12-02
2,0.0,Stage_5,2019-12-05
3,0.0,Stage_2,2019-12-11
4,0.0,Stage_7,2019-12-17


In [45]:
import time
t = time.time()
CASE_ID_COLNAME = 'id'
ACTIVITY_ID_COLNAME = 'stages'
TIMESTAMP_COLNAME = 'dt'
log = conversion_factory.apply(df, parameters={'case_id_glue': CASE_ID_COLNAME})

parameters = {'pm4py:param:activity_key': ACTIVITY_ID_COLNAME,
                  'dependency_thresh': 0.95,
                  'min_act_count': 1,
                  'min_dfg_occurrences': 1}
heu_net = heuristics_miner.apply_heu(log, parameters=parameters)
gviz_params = {"format": "png"}
               # "splines": "curved",
               # "quadType": "fast",
               # "engine": "neato",
               # "layout": "sfpd"}
name = 'image'
gviz = hn_vis_factory.apply(heu_net, parameters=gviz_params)
hn_vis_factory.save(gviz, 'heu_net_{}.png'.format(name))
print(time.time() - t)

Init 0.0
1 loop 0.0009915828704833984
2 loop 0.0
3 loop 0.0004963874816894531
4 loop 0.0009920597076416016
Last loop 0.0014874935150146484

Write graph 0.09123420715332031
0.13536524772644043


In [48]:
class process_mining:
    def __init__(self, df,threshold):
        self.data = df
        self.stages = np.sort(df.stages.unique())
        self.stages = np.insert(self.stages,0,"Start")
        self.stages = np.insert(self.stages,len(self.stages),"End")    
        self.l = self.stages.shape[0]
        self.matrix = np.zeros((self.l,self.l))        
        self.bin_matrix = np.zeros((self.l,self.l))
        self.indexs = []
        self.initial_state = ''
        self.initial_index = 0
        self.threshold = threshold
        self.id = pd.unique(df['id']) 
    #считает количетсво случаев когда stage1 идёт после stage2 и наоборот  
    def numbers_of_cases(self,stage1,stage2):
        ar_1=self.data[(self.data.stages==stage1)]
        ar_2=self.data[(self.data.stages==stage2)]
        ar=ar_1.merge(ar_2,left_on='id', right_on='id')        
        #1 значение - кол-во идущих после
        #2 значение - кол-во идущих раньше
        return [ar[ar.dt_x>ar.dt_y].shape[0],ar[ar.dt_x<ar.dt_y].shape[0]]

    #считает коэффициент (1) описанный в статье, который нужно записать в матрицу
    def rate(self,rate1,rate2):
        return (rate1-rate2)/(rate1+rate2+1)
        
    #вычисляет матрицу переходных вероятностей
    def matrix_prob(self):
        for i in range(1,self.l-1):
            self.matrix[0][i] = self.data[(self.data.stages==self.stages[i])].shape[0]/self.data.shape[0]
        for i in range(1,self.l-1):
            for j in range(1,self.l-1):                
                self.matrix[i][j]=self.rate(*self.numbers_of_cases(self.stages[i],self.stages[j]))

        self.matrix[self.l-1] = np.abs(self.matrix.sum(axis = 1)/self.l)/np.abs(self.matrix.sum(axis = 1)/self.l).sum()        
        return self.matrix
    
    #бинаризируем по порогуth как в статье
    def matrix_bin(self):
        self.bin_matrix[self.matrix>=self.threshold] = 1
        self.bin_matrix[(self.matrix<self.threshold)] = 0
        return self.bin_matrix
        
    #строим граф по matrix_bin
    def matrix_for_graph(self):
        self.indexs=[]
        index=np.all(self.bin_matrix==0,axis=1)
        self.initial_state=self.stages[index]
        self.initial_index=np.argwhere(index == True)
        self.recur_func(*np.argwhere(index== True))
        return self.indexs
    
    #идём по графу процесса из вершины в вершину по маьрице смежности и записываем название рёбер для отрисовки графа
    def recur_func(self,*nodes):
        nodes_new=[]
        ch=0
        for node in nodes:
            for j in node:
                for i in range(self.l):
                    if(self.bin_matrix[i,j]):
                        ch=1
                        self.indexs.extend([str(j)+"-"+str(int(i))])
                        nodes_new.append(i)
                break
        if ch==1:
            self.recur_func(nodes_new)
        else:
            return 0 
    #отрисовка графа
    def graph(self):
        self.matrix_prob()
        self.matrix_bin()
        self.matrix_for_graph()
        
        dot = Digraph(comment='Process',format = 'pdf')
        dot.attr(rankdir='Stages',size='8,5')
        
        #nodes        
        #задаём начальную ноду отдельно
        for name, init in zip(self.initial_index,self.initial_state):
            dot.node(init)                
        ind=np.any(self.bin_matrix>0,axis=1)        
        for i in range(self.l):
            if ind[i]:
                dot.node(self.stages[i])
        
        #edges
        for i in range(1,len(self.stages)-1):
            dot.edge(self.stages[0],self.stages[i],
                     label = str(float('{:.3f}'.format(self.matrix[0][int(i)]))))
            
            dot.edge(self.stages[i],self.stages[len(self.stages)-1],
                     label = str(float('{:.3f}'.format(self.matrix[self.l-1][int(i)]))))
            
        for i in range(len(self.indexs)):
            a,b = self.indexs[i].split('-')
            dot.edge(self.stages[int(a)],
                      self.stages[int(b)],
                    label = str(float('{:.3f}'.format(self.matrix[int(a)][int(b)]*(-1)))))
            
        dot.format = 'pdf'
        #dot.render()
        return dot
        
    # get методы для удобной работы
    def get_matrix(self):
        return self.matrix_prob()
    
    def get_matrix_bin(self):
        self.matrix_prob()
        return self.matrix_bin()
    
    def get_matrix_graph(self):
        self.matrix_prob()
        self.matrix_bin()
        return self.matrix_for_graph()
    

In [15]:
all_events = pd.unique(df['stages'])
begin_events = 
end_events = 

array(['Stage_0', 'Stage_3', 'Stage_5', 'Stage_2', 'Stage_7', 'Stage_4',
       'Stage_8', 'Stage_9', 'Stage_6', 'Stage_1'], dtype=object)

In [49]:
pm=process_mining(df,0.95)
gr = pm.graph()
gr.view()

'Digraph.gv.pdf'

In [25]:
pd.DataFrame(pm.matrix)

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.000000,0.169917,0.047747,0.150639,0.129343,0.039901,0.137413,0.024658,0.110962,0.141896,0.047523,0.0
1,0.000000,0.000000,-0.670807,-0.732673,-0.825792,-0.895833,-0.926503,-0.964706,-0.976000,-0.981405,-0.993976,0.0
2,0.000000,0.670807,0.000000,-0.682432,-0.869565,-0.909091,-0.932836,-0.958333,-0.990476,-0.976744,-0.975610,0.0
3,0.000000,0.732673,0.682432,0.000000,-0.696429,-0.808333,-0.868106,-0.853333,-0.919881,-0.969977,-0.992754,0.0
4,0.000000,0.825792,0.869565,0.696429,0.000000,-0.648936,-0.686441,-0.857143,-0.882943,-0.937330,-0.961832,0.0
5,0.000000,0.895833,0.909091,0.808333,0.648936,0.000000,-0.712963,-0.450000,-0.876404,-0.915888,-0.962963,0.0
6,0.000000,0.926503,0.932836,0.868106,0.686441,0.712963,0.000000,-0.738462,-0.749186,-0.873385,-0.891304,0.0
7,0.000000,0.964706,0.958333,0.853333,0.857143,0.450000,0.738462,0.000000,-0.685185,-0.723684,-0.941176,0.0
8,0.000000,0.976000,0.990476,0.919881,0.882943,0.876404,0.749186,0.685185,0.000000,-0.755700,-0.930693,0.0
9,0.000000,0.981405,0.976744,0.969977,0.937330,0.915888,0.873385,0.723684,0.755700,0.000000,-0.744526,0.0
